In [1]:
from reader import ebook
from main import read_list,read_json,get_images
from typing import Optional
import requests
from dotenv import load_dotenv
import json
import os


In [2]:
load_dotenv()
API = os.getenv("HF_API")

headers = {
    "Authorization": f"Bearer {API}",
    "Content-Type": "application/json",
}
url = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3/v1/chat/completions"


In [15]:
role='''(NOTE: Only output in JSON. Ensure the JSON format is valid, well-formed, and easy to parse.)
Input:  
1.Chapter Text: The current chapter to be analyzed.
2.Character List: A list of characters with their descriptions till now (This chapter).
3.Places list: list of places and their description till now (This chapter).
3.Previous Chapters' Summary: Context from earlier chapters.

Rules:  
1.Narrative Summary: Summarize the chapter in detail , integrating context and key developments from previous chapters and create a self containing summary. end with to be continued.
2.Character List: add new characters to the list based on this chapter and Update existing character descriptions. If no characters are mentioned, return the same list as given.
3.Places: Include an updated description of any significant locations mentioned in this chapter, focusing on environment, weather, vibe, and structure.
4.Output Format: Ensure the output is valid and well-structured JSON.

Output:  
Generate a JSON object in this format:
{
  "summary": "Detailed Summary of the current chapter in context of previous chapters.",
  "characters": {
      "Character Name": "Updated or new description (age, looks, clothes, hair, body language) based on this chapter."
    },
  "places": {
      "Place Name": "Updated or new description (environment, weather, vibe, structure, etc.) based on this chapter."
  }
}

example:
{
  "summary": "The chapter opens with the protagonist, Monsieur Meursault, receiving a telegram about his mother's death. 
  Despite the news, he reacts with emotional detachment, pondering the ambiguity of the exact day of her passing.
  He prepares to attend the funeral, arranging leave from work, but notes his employer's subtle annoyance.
  His journey to the Home for Aged Persons in Marengo reveals the intense heat of the Algerian climate and his interactions with others, 
  such as a soldier on the bus and the doorkeeper. 
  At the Home, Meursault encounters the warden, who seems both judgmental and understanding about the circumstances leading to his mother's stay. 
  The chapter concludes with Meursault entering the mortuary, where his lack of emotion continues as he declines to view the body, further emphasizing his detachment.",
  
  "characters": {
    "Monsieur Meursault": "A young man, emotionally detached and indifferent. He appears practical, with a reserved demeanor. His body language reflects discomfort when dealing with social interactions, such as prolonged handshakes or unnecessary apologies.",
    "The Doorkeeper": "A man in his sixties, hale and hearty with a cheerful demeanor. He has ruddy cheeks, blue eyes, and a white mustache. Though officially a doorkeeper, he sees himself as an authority figure among the residents.",
    "The Warden": "A small, older man with gray hair and watery blue eyes. He wears a Legion of Honor rosette and has a formal yet slightly judgmental attitude. He holds a prolonged handshake, perhaps signaling awkwardness or an attempt at connection."
  },
  "places": {
    "The Home for Aged Persons": "Located in Marengo, fifty miles from Algiers, the Home has a formal and slightly oppressive atmosphere. The environment is marked by a clean mortuary, whitewashed walls, and a courtyard where elderly residents gather. The weather is intensely hot, reflecting the harsh Algerian climate.",
    "Marengo": "A small village with a rural, subdued atmosphere. The intense afternoon heat creates a sense of lethargy and discomfort. The Home itself is slightly isolated, requiring a mile-long walk from the bus stop."
  }
}

'''


In [16]:
def msg(text: str, context: str = "Its first chapter", characters: dict = {}, places: dict = {}) -> list:
    js=json.dumps({
                "past_context": context,
                "text": text,
                "character_list": characters,
                "places_list": places})
    message = [
        {
            "role": "system",
            "content": role
        },
        {
            "role": "user",
            "content": json.dumps({
                "past_context": context,
                "text": text,
                "character_list": characters,
                "places_list": places
            }),
        },
    ]
    return message

In [5]:
def get_scene(messages) -> Optional[str]:
    data = {
        "messages": messages,
        "temperature": 0.7,
        "stream": False,
        "max_tokens":10000
    }
    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        response_data = response.json()
        assistant_message = response_data["choices"][0]["message"]["content"]
        return assistant_message
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

In [17]:
book=ebook("./books/LP.epub")
chapters = book.get_chapters()
_,text=zip(*chapters[:10]) 



In [19]:

output_dict={
    0:
        {
            "summary":"This is the first chapter",
            "characters":{},
            "places": {}
        }
            }

In [20]:
for idx,i in enumerate(text):
    mes=msg(text,output_dict[idx]["summary"],characters=output_dict[idx]["characters"],places=output_dict[idx]["places"])
    summary_characters=get_scene(mes)
    output_json=read_json(summary_characters)
    output_dict[idx+1]=output_json
    print(f"chapter: {idx} done")
    

chapter: 0 done
chapter: 1 done
chapter: 2 done
chapter: 3 done
chapter: 4 done
chapter: 5 done
chapter: 6 done
chapter: 7 done
chapter: 8 done
chapter: 9 done


In [ ]:
output_dict

In [21]:
for num, dic in output_dict.items():
    if num==0:
        continue
    print(f"CHAPTER {num} :\n")
    
    sum,char,places=dic.items()
    print("Characters:")
    if len(char)>0 :
        for name,i in char[1].items():
            print(f"{name}: {i}")
    
    print("\nPlaces:")
    if len(places)>0:
        for name,i in places[1].items():
            print(f"{name}: {i}")
    print(f"\nSummary:\n{sum[1]}")
    print("\n-------------------------------------------------------------------------------------------------------------------")

CHAPTER 1 :

Characters:
The Storyteller: A grown-up who has encountered the Little Prince in the Sahara Desert. He shares the Little Prince's stories and reflects on their encounter.
The Little Prince: A mysterious character who comes from a planet with stars as his companions. He shares stories of his own planet and its challenges.

Places:
The Sahara Desert: A vast, isolated desert where the storyteller encounters the Little Prince.
The Little Prince's Planet: A unique planet inhabited by stars, baobabs, and the Little Prince's flower.

Summary:
In this chapter, the storyteller reflects on his childhood and the encounter with the Little Prince, a mysterious character who came to him in the Sahara Desert. The Little Prince shares stories of his own planet, where he lived among stars and faced challenges such as the drama of the baobabs and the complexities of love with a rose. The storyteller learns about the Little Prince's flower, which he had nurtured and cared for but ultimately 

In [25]:
for key, val in output_dict.items():
    print(val["summary"])

This is the first chapter
In this chapter, the storyteller reflects on his childhood and the encounter with the Little Prince, a mysterious character who came to him in the Sahara Desert. The Little Prince shares stories of his own planet, where he lived among stars and faced challenges such as the drama of the baobabs and the complexities of love with a rose. The storyteller learns about the Little Prince's flower, which he had nurtured and cared for but ultimately had to leave behind. The chapter ends with the Little Prince's departure from the storyteller's planet, leaving him with a sense of loss and longing.
The chapter delves into the storyteller's past, recounting his encounter with the mystical Little Prince in the Sahara Desert. The Little Prince shares tales of his home planet, where he lives among stars and faces challenges like the drama of the baobabs and the complexities of love with a rose. The storyteller learns about the Little Prince's flower, which he had nurtured an

In [407]:
b

{'narrative_summary': 'The narrator recounts his past experiences, including the loss of his artistic dreams due to the lack of understanding from adults, and his encounter with a mysterious voice in the Sahara Desert that asked him to draw a sheep. This encounter rekindled his connection to childhood innocence.',
 'characters': {'The Narrator': 'An unnamed individual who is a seasoned traveler and pilot. He has a cynical view of adults and their lack of imagination. He carries a drawing of a mysterious little man he encountered in the Sahara Desert.'}}

In [402]:
char=b['characters']
summary=b['narrative_summary']

In [404]:
print("characters:")
for name,disc in char.items():
    print(f"{name}:\n{disc}")
print(f"\nSummary:\n{summary}")


characters:
The Narrator:
An unnamed character who is a grown-up, a pilot, and an artist at heart. He has a pessimistic view of adults and their lack of imagination. He wears a pilot's uniform and has a serious expression.

Summary:
The narrator reflects on his past, his lost artistic dreams, and his encounter with a mysterious voice in the Sahara Desert. This voice, later identified as the little prince, asks him to draw a sheep, reigniting his connection to childhood innocence.
